In [3]:
import bitstring as bs
import numpy as np

In [4]:
def next_float(f):
    #packing double-precision foat
    b = bs.pack('>d', f)
    
    #extracting mantisa as unsigned int and adding up 1.
    # There are two cases, (1) if the bits of the mantissa are all 1,
    # (2) all the other cases.
    m = b[12:].uint
    if m==4503599627370495:
        # Case (1)
        m=0
        b[12:] = m
        exp=b[1:12].uint
        exp +=1
        b[1:12] = exp
        
    else:
        # Case (2)
        m += 1
        # putting the result in his place
        b[12:] = m
    
    return b.float

def to_binary(f):
    b = bs.pack('>d', f)
    b = b.bin
    #show sign + exponent + mantisa
    print(b[0]+' '+b[1:12]+' '+b[12:])

def all_equal(iterator):
    iterator = iter(iterator)
    first = next(iterator)
    return (all(first == x for x in iterator), first)

In [40]:
def f_new_rep(x,bits_mant,bits_exp):
    # Algoritmo de representación de punto flotante modificada.
    n = bs.pack('>d', x) # Pasar el numero original a punto flotante
    b = n.bin

    # Signo
    signo = b[0]

    # Trabajar la mantisa
    mantisa = n[12:12+bits_mant]
    mantisa_b = n[12:12+bits_mant].uint
    if bits_mant < 52: # Si no se necesita la mantisa original completa
        if n[12 + bits_mant - 1] == '1' or int('0b' + b[bits_mant+12:], 2) > 2**(51-bits_mant):
            if mantisa_b == 2**bits_mant - 1:
                #print('1')
                mantisa_b = 0
                n[12:12+bits_mant] = mantisa_b
                exp = n[1:12].uint
                exp += 1
                n[1:12] = exp
            else:
                #print('2')
                mantisa_b += 1
                n[12:12+bits_mant] = mantisa_b
                #print(n[12:12+bits_mant])
    mantisa = n[12:12+bits_mant].bin

    # Trabajar el exponente nuevo 
    exp = int('0b' + b[1:12], 2) - 1023 + ((2**(bits_exp-1))-1) # Calcular el valor decimal del nuevo exponente 
    exp = bin(exp)[2:] # Pasarlo a binario nuevamente
    exp = (bits_exp-len(exp))*'0' + exp # Agregarle 0's si hace falta

    #print(signo + ' ' + exp + ' ' + mantisa)  # Juntar todo

    n[12+bits_mant:] = 0

    #b = n.bin
    #print(b[0]+' '+b[1:12]+' '+b[12:])  # Juntar todo
    
    result = 0

    # Revisar casos especiales
    equal = all_equal(exp) 
    #print(len(exp))
    if len(exp) > bits_exp:
        result = np.inf
    else:
        if equal[0]:
            if equal[1] == '1':
                if signo == '0':
                    result = np.inf
                else:
                    if all_equal(n[12:12+bits_mant])[0]:
                        result = -np.inf
                    else:
                        result = np.nan
        else:
            result = n.float

    return  result

In [41]:
# 1.2343
x = 1.2343
a = 9819824.624837
to_binary(a)
f_new_rep(a, 29, 4)

0 10000010110 0010101110101101011000010011111111101010101000101010


inf

In [42]:
def f_suma(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la suma de dos numeros manteniendo el estandar asignado
    result = f_new_rep(n1 + n2, bits_mant, bits_exp) 
    return result

def f_multiplicacion(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la multiplicación de dos numeros manteniendo el estandar asignado
    result = f_new_rep(n1 * n2, bits_mant, bits_exp) 
    return result

def f_division(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la división de dos numeros manteniendo el estandar asignado
    if n2 == 0:
        if n1 == 0:
            return np.nan
        else:
            return np.inf
    else:
        result = f_new_rep(n1/n2, bits_mant, bits_exp)
        return result

def f_find_roots(A,bits_mant,bits_exp):
    # Algoritmo para encontrar raíces de f(x).
    return x_roots

In [43]:
# Algoritmo de ejemplo
a = 9819824.624837
b = 148736.523476
c = a+b

bits_mant = 20
for i in [6, 5, 4, 3, 2]:
    bits_exp = i
    bits_mant = 20
    # Implementación de ejemplo con Representación de Punto Flotante de 'bits_mant' bits en la mantisa y 'bits_exp' bits en el exponente.
    am = f_new_rep(9819824.624837,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
    bm = f_new_rep(148736.523476,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
    cm = f_suma(am,bm,bits_mant,bits_exp) # Aproximar el output de la suma y cada operación en la nueva representación.
    print('bits_exp =', bits_exp, "    bits_mant =", bits_mant)
    print("original         new_rep")
    print(a, ' ', am)
    print(b, '  ', bm)
    print(c, ' ', cm)
    print("-------------------------------------------")

bits_exp = 6     bits_mant = 20
original         new_rep
9819824.624837   9819824.0
148736.523476    148736.5
9968561.148313   9968560.0
-------------------------------------------
bits_exp = 5     bits_mant = 20
original         new_rep
9819824.624837   inf
148736.523476    inf
9968561.148313   inf
-------------------------------------------
bits_exp = 4     bits_mant = 20
original         new_rep
9819824.624837   inf
148736.523476    inf
9968561.148313   inf
-------------------------------------------
bits_exp = 3     bits_mant = 20
original         new_rep
9819824.624837   inf
148736.523476    inf
9968561.148313   inf
-------------------------------------------
bits_exp = 2     bits_mant = 20
original         new_rep
9819824.624837   inf
148736.523476    inf
9968561.148313   inf
-------------------------------------------
